In [1]:
# %%capture
# Capture because braindecode module prints a lot :S
from pyFiles.training import *
from pyFiles.models import *
from pyFiles.dataprep_down import *
from pyFiles.dataprep2_down import *
#
import torch
from torch import nn
from torch.nn import functional as F
#
from pyFiles.losses import InfoNceLoss, VICReg, BarlowTwins

c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_outputs' is missing.
  self._warn(
c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_chans' is missing.
  self._warn(
c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [3]:
#PREP DENUEVO
# prep("C:\\Users\\TheSy\\Desktop\\dataverse_files", mode = "per_win", save = True, save_dir = "data_down")

In [4]:
# spliter = DFSpliter(save = True, mode= "down_win")
# spliter("down_prep_windows.csv",".")

In [2]:
transforms = Augmentations(5, False, AUGMENTATIONS)

In [3]:
p_size = 32
class CustomEEGDataset(Dataset):
    def __init__(self, csv_file , root_dir , transform = None, ):

        try:
            self.loc_df = pd.read_csv(os.path.join(root_dir,csv_file)).drop(labels="Unnamed: 0", axis = 1)
        except:
            self.loc_df = pd.read_csv(os.path.join(root_dir,csv_file))

        self.transform = transform
        self.root_dir = root_dir

    def __len__(self,):
        return len(self.loc_df)//p_size
    

    def __getitem__(self, idx):
        exam_shape = (p_size, 19, 1000)
        # print(val_shape)
            # return ch, diagnostic
        new_exam = torch.ones(exam_shape)
        # for i in range(len(self.loc_df)):
        for j in range(p_size):
            eeg_file = os.path.join(self.root_dir,
                            self.loc_df.iloc[idx*p_size + j, 2])
            ch, diagnostic = torch.load(eeg_file)
        
            if self.transform is not None:
                ch = self.transform(ch)
            new_exam[j] = ch
        
        return new_exam, diagnostic

In [4]:
train_dataset = CustomEEGDataset("down_win_train_feats.csv", root_dir = "", transform = transforms)
val_dataset = CustomEEGDataset("down_win_val_feats.csv", root_dir = "",)

In [5]:
# trainloader = DataLoader(train_dataset, batch_size = 1, shuffle = True, num_workers = 0)
trainloader = DataLoader(train_dataset, batch_size = 22, shuffle = True, num_workers = 0)
valloader = DataLoader(val_dataset, batch_size = 12, shuffle = False, num_workers = 0)

In [6]:
model = SleepStager(n_chans = 19, pretext = False, p_size = 32)
enc, fc = torch.load("params\multis\VR\\attempt2\\bestEncoderParamsmulti_VR.pt")
# enc, fc = torch.load("bestEncoderParamsmulti_VR.pt")
# model.encoder.load_state_dict(enc)

criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()

#FREEZE
# for param in model.encoder.parameters():
#   param.requires_grad = False

optimizer = torch.optim.Adam(
    [
        {"params": model.encoder.parameters(), "lr": 1e-8},
        {"params": model.final.parameters(), "lr": 5e-4},
    ],
    lr=5e-4,
) 
downtrain(model, 60, trainloader, valloader, criterion, optimizer, state = None, name = "multi_vr")

Epoch nro 1/60
Iter: 1/2, Loss:0.7107014060020447
Iter: 2/2, Loss:0.7561836838722229
Epoch 1/60: Train loss = 0.7334425449371338, Val loss = 0.7034016251564026, Train acc = 0.4772727272727273, Val acc = 0.5
Better params found in epoch = 1, saved params
Epoch nro 2/60
Iter: 1/2, Loss:0.7039861679077148
Iter: 2/2, Loss:0.7169392108917236
Epoch 2/60: Train loss = 0.7104626893997192, Val loss = 0.7148202061653137, Train acc = 0.5, Val acc = 0.5
Epoch nro 3/60
Iter: 1/2, Loss:0.7572538256645203
Iter: 2/2, Loss:0.6246896386146545
Epoch 3/60: Train loss = 0.6909717321395874, Val loss = 0.6752925515174866, Train acc = 0.6136363636363636, Val acc = 0.5
Better params found in epoch = 3, saved params
Epoch nro 4/60
Iter: 1/2, Loss:0.6113432049751282
Iter: 2/2, Loss:0.6941144466400146
Epoch 4/60: Train loss = 0.6527288258075714, Val loss = 0.6682057976722717, Train acc = 0.6136363636363636, Val acc = 0.5
Better params found in epoch = 4, saved params
Epoch nro 5/60
Iter: 1/2, Loss:0.6984147429466

KeyboardInterrupt: 

In [31]:
model

SleepStager(
  (encoder): SleepStagerChambon2018(
    (spatial_conv): Conv2d(1, 19, kernel_size=(19, 1), stride=(1, 1))
    (feature_extractor): Sequential(
      (0): Conv2d(1, 16, kernel_size=(1, 50), stride=(1, 1), padding=(0, 25))
      (1): Identity()
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(1, 13), stride=(1, 13), padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 16, kernel_size=(1, 50), stride=(1, 1), padding=(0, 25))
      (5): Identity()
      (6): ReLU()
      (7): MaxPool2d(kernel_size=(1, 13), stride=(1, 13), padding=0, dilation=1, ceil_mode=False)
    )
  )
  (final): RNNClassifier(
    (rnn): RNN(1824, 100, batch_first=True)
    (fc): Linear(in_features=100, out_features=2, bias=True)
  )
)